## Assignment

Don't forget to apply the most suitable missing value filling techniques you applied in the previous checkpoints to the data. You should provide your answers to the following questions after you handled the missing values.

Say, we want to understand the relationship between the expenditures of the governments and the students' overall success in the math and reading.

1) Create a new score variable from the weighted averages of all score variables in the datasets. 
    Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade.       So, you should appropriately weigh the scores!.

2) What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

3) Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

4) What is the correlation between the overall score variable and the 1st principal component?

5) If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

In [1]:
import pandas as pd
import numpy as np
import statistics
import matplotlib.pyplot as plt
%matplotlib inline
import scipy.stats as stats
from scipy.stats.mstats import winsorize
import seaborn as sns
from sqlalchemy import create_engine
import warnings
import sklearn
from sklearn.preprocessing import normalize
from sklearn.preprocessing import scale

warnings.filterwarnings('ignore')
sns.set(style="whitegrid")


In [2]:
#Import dataset from postgres sql server

#defines the logon credentials
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

#opens the connection
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(postgres_user, postgres_pw, postgres_host,
    postgres_port, postgres_db))

#queries the schema
education_df = pd.read_sql_query('select * from useducation', con=engine)

#closes the connection
engine.dispose()

In [3]:
education_df.head(10)

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
0,1992_ALABAMA,ALABAMA,1992,NaN,2678885.0,304177.0,1659028.0,715680.0,2653798.0,1481703.0,...,57948.0,58025.0,41167.0,471564.0,196386.0,676174.0,208.327876,252.187522,207.963517,NaN
1,1992_ALASKA,ALASKA,1992,NaN,1049591.0,106780.0,720711.0,222100.0,972488.0,498362.0,...,9748.0,8789.0,6714.0,79117.0,30847.0,112335.0,NaN,NaN,NaN,258.859712
2,1992_ARIZONA,ARIZONA,1992,NaN,3258079.0,297888.0,1369815.0,1590376.0,3401580.0,1435908.0,...,55433.0,49081.0,37410.0,437127.0,175210.0,614881.0,215.253932,265.366278,206.212716,262.169895
3,1992_ARKANSAS,ARKANSAS,1992,NaN,1711959.0,178571.0,958785.0,574603.0,1743022.0,964323.0,...,34632.0,36011.0,27651.0,281338.0,123113.0,405259.0,210.206028,256.312090,208.634458,264.619665
4,1992_CALIFORNIA,CALIFORNIA,1992,NaN,26260025.0,2072470.0,16546514.0,7641041.0,27138832.0,14358922.0,...,418418.0,363296.0,270675.0,3286034.0,1372011.0,4717112.0,208.398961,260.892247,196.764414,NaN
5,1992_COLORADO,COLORADO,1992,NaN,3185173.0,163253.0,1307986.0,1713934.0,3264826.0,1642466.0,...,50648.0,45025.0,34533.0,394904.0,160299.0,562613.0,221.023429,272.398433,213.480890,266.419687
6,1992_CONNECTICUT,CONNECTICUT,1992,NaN,3834302.0,143542.0,1342539.0,2348221.0,3721338.0,2148041.0,...,38058.0,33691.0,28366.0,304284.0,126917.0,436932.0,226.798480,273.739345,222.417599,264.713904
7,1992_DELAWARE,DELAWARE,1992,NaN,645233.0,45945.0,420942.0,178346.0,638784.0,372722.0,...,8272.0,8012.0,6129.0,67495.0,28338.0,96296.0,217.899972,262.868585,206.389052,261.577921
8,1992_DISTRICT_OF_COLUMBIA,DISTRICT_OF_COLUMBIA,1992,NaN,709480.0,64749.0,0.0,644731.0,742893.0,329160.0,...,5832.0,5000.0,3433.0,47009.0,18173.0,70000.0,192.600553,234.924621,178.557612,265.976765
9,1992_FLORIDA,FLORIDA,1992,NaN,11506299.0,788420.0,5683949.0,5033930.0,11305642.0,5166374.0,...,164416.0,142372.0,100835.0,1276685.0,511557.0,1819706.0,213.689751,259.908848,204.882606,262.118986


In [4]:
education_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1492 entries, 0 to 1491
Data columns (total 25 columns):
PRIMARY_KEY                     1492 non-null object
STATE                           1492 non-null object
YEAR                            1492 non-null int64
ENROLL                          1229 non-null float64
TOTAL_REVENUE                   1280 non-null float64
FEDERAL_REVENUE                 1280 non-null float64
STATE_REVENUE                   1280 non-null float64
LOCAL_REVENUE                   1280 non-null float64
TOTAL_EXPENDITURE               1280 non-null float64
INSTRUCTION_EXPENDITURE         1280 non-null float64
SUPPORT_SERVICES_EXPENDITURE    1280 non-null float64
OTHER_EXPENDITURE               1229 non-null float64
CAPITAL_OUTLAY_EXPENDITURE      1280 non-null float64
GRADES_PK_G                     1319 non-null float64
GRADES_KG_G                     1360 non-null float64
GRADES_4_G                      1361 non-null float64
GRADES_8_G                      1

### 1) Clean data and add new variables

Don't forget to apply the most suitable missing value filling techniques you applied in the previous checkpoints to the data. You should provide your answers to the following questions after you handled the missing values.

In [5]:
#Create a list called 'names' 
names = ['ENROLL','TOTAL_REVENUE',                   
'FEDERAL_REVENUE',                 
'STATE_REVENUE',                   
'LOCAL_REVENUE',                   
'TOTAL_EXPENDITURE',              
'INSTRUCTION_EXPENDITURE',        
'SUPPORT_SERVICES_EXPENDITURE',    
'OTHER_EXPENDITURE',              
'CAPITAL_OUTLAY_EXPENDITURE',      
'GRADES_PK_G',                   
'GRADES_KG_G',                     
'GRADES_4_G',                      
'GRADES_8_G',                      
'GRADES_12_G',                     
'GRADES_1_8_G',                    
'GRADES_9_12_G',                   
'GRADES_ALL_G']

#Then iterate using for loop for each column to fill mean for NaN values

for name in names:
    education_df["{}".format(name)]=education_df.groupby(["YEAR"])["{}".format(name)]\
        .transform(lambda x : x.fillna(x.mean()))

In [6]:
#Then iterate using for loop for each column to fill NaN values using interlopation

for name in names:
    education_df["{}".format(name)]=education_df["{}".format(name)]\
        .interpolate(method='nearest', limit_direction='backward')

In [7]:
#Now drop any remaining zero or missing values
education_df.dropna(inplace=True)

In [8]:
education_df.describe(include='all')

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_4_G,GRADES_8_G,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE
count,422,422,422.000000,4.220000e+02,4.220000e+02,4.220000e+02,4.220000e+02,4.220000e+02,4.220000e+02,4.220000e+02,...,422.000000,422.000000,422.000000,4.220000e+02,4.220000e+02,4.220000e+02,422.000000,422.000000,422.000000,422.000000
unique,421,51,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,2009_DISTRICT_OF_COLUMBIA,DISTRICT_OF_COLUMBIA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,2,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,2007.649289,9.373755e+05,1.042616e+07,9.382971e+05,4.829200e+06,4.658659e+06,1.053020e+07,5.443115e+06,...,71669.156398,72295.779621,63690.436019,5.776714e+05,2.833822e+05,8.734688e+05,236.609257,279.546159,219.316208,263.564880
std,NaN,NaN,5.369062,1.113565e+06,1.299493e+07,1.316457e+06,6.112630e+06,6.041350e+06,1.323610e+07,6.942162e+06,...,85674.615405,85324.072358,76294.727761,6.879329e+05,3.377391e+05,1.023265e+06,8.950257,9.541264,7.536608,6.907147
min,NaN,NaN,1996.000000,4.417900e+04,6.640570e+05,4.100700e+04,0.000000e+00,2.853100e+04,6.763050e+05,3.141380e+05,...,3423.000000,2860.000000,2188.000000,2.676200e+04,9.970000e+03,3.682300e+04,187.134670,232.831510,181.626990,236.379102
25%,NaN,NaN,2003.000000,2.794832e+05,2.663214e+06,2.593982e+05,1.420028e+06,9.432548e+05,2.624100e+06,1.403394e+06,...,20433.500000,20792.000000,18503.250000,1.658642e+05,8.163050e+04,2.596278e+05,231.971719,274.474773,214.951746,259.425389
50%,NaN,NaN,2008.000000,6.659445e+05,6.087984e+06,5.408265e+05,2.980016e+06,2.386380e+06,6.197553e+06,3.171867e+06,...,50653.500000,50114.500000,41194.000000,4.055395e+05,1.861810e+05,6.282085e+05,238.229823,281.369105,220.823937,265.070052
75%,NaN,NaN,2013.000000,1.012553e+06,1.185052e+07,1.013968e+06,5.955812e+06,5.190645e+06,1.142614e+07,5.965888e+06,...,78853.500000,80820.000000,74694.750000,6.378688e+05,3.199545e+05,9.670040e+05,242.602566,286.094495,224.412843,268.199808


### 2) Create a new score variable from the weighted averages of all score variables in the datasets. 
    Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade.       So, you should appropriately weigh the scores!.

In [9]:
print('4th grade students = ', education_df.groupby('YEAR').sum()['GRADES_4_G'])
print('8th grade students = ', education_df.groupby('YEAR').sum()['GRADES_8_G'])

4th grade students =  YEAR
1996    1897323.0
2000    2419569.0
2003    3619089.0
2005    3577506.0
2007    3624382.0
2009    3703953.0
2011    3671865.0
2013    3707733.0
2015    3792735.0
2017     230229.0
Name: GRADES_4_G, dtype: float64
8th grade students =  YEAR
1996    1849599.0
2000    2295829.0
2003    3809431.0
2005    3801893.0
2007    3708955.0
2009    3654136.0
2011    3679111.0
2013    3753357.0
2015    3718562.0
2017     237946.0
Name: GRADES_8_G, dtype: float64


In [19]:
#weighted avg score for both 4th & 8th grade
education_df["AVG_SCORE"] = (education_df["GRADES_4_G"]*((education_df["AVG_MATH_4_SCORE"] + education_df["AVG_READING_4_SCORE"])*0.5) + education_df["GRADES_8_G"]
                                 * ((education_df["AVG_MATH_8_SCORE"] + education_df["AVG_READING_8_SCORE"])*0.5))/(education_df["GRADES_4_G"] + education_df["GRADES_8_G"])


In [20]:
education_df.head(10)

,PRIMARY_KEY,STATE,YEAR,ENROLL,TOTAL_REVENUE,FEDERAL_REVENUE,STATE_REVENUE,LOCAL_REVENUE,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,...,GRADES_12_G,GRADES_1_8_G,GRADES_9_12_G,GRADES_ALL_G,AVG_MATH_4_SCORE,AVG_MATH_8_SCORE,AVG_READING_4_SCORE,AVG_READING_8_SCORE,avg_score,AVG_SCORE
209,1996_ALABAMA,ALABAMA,1996,735912.0,3365113.0,334858.0,2138717.0,891538.0,3572283.0,1987018.0,...,42510.0,469370.0,207980.0,686983.0,211.646974,256.594863,210.920767,260.276441,235.106342,235.106342
211,1996_ARIZONA,ARIZONA,1996,764681.0,4143304.0,378917.0,1798363.0,1966024.0,4391555.0,1860007.0,...,42041.0,511925.0,210289.0,726709.0,217.575940,267.874834,206.529788,263.283638,238.321404,238.321404
212,1996_ARKANSAS,ARKANSAS,1996,452907.0,2183384.0,192152.0,1296247.0,694985.0,2297381.0,1264275.0,...,27613.0,283883.0,131997.0,417860.0,215.846436,261.652745,208.837716,271.834590,240.773675,240.773675
213,1996_CALIFORNIA,CALIFORNIA,1996,5460484.0,31282981.0,2603882.0,17061474.0,11617625.0,31561692.0,16880730.0,...,298669.0,3516572.0,1525618.0,5115423.0,209.129083,262.772478,201.912591,269.197451,234.719834,234.719834
214,1996_COLORADO,COLORADO,1996,655679.0,3891203.0,194998.0,1664676.0,2031529.0,4075165.0,2052264.0,...,37179.0,423510.0,185700.0,621730.0,225.805858,275.608214,221.653243,268.168154,247.750273,247.750273
215,1996_CONNECTICUT,CONNECTICUT,1996,499815.0,4686025.0,162218.0,1703171.0,2820636.0,4538234.0,2701637.0,...,28882.0,335296.0,137753.0,482196.0,232.026574,279.591245,231.598305,263.689781,250.735755,250.735755
217,1996_DISTRICT_OF_COLUMBIA,DISTRICT_OF_COLUMBIA,1996,79802.0,678874.0,57870.0,0.0,621004.0,703245.0,334893.0,...,3042.0,46313.0,16241.0,67598.0,187.134670,232.831510,181.626990,264.938466,214.015451,214.015451
218,1996_FLORIDA,FLORIDA,1996,2175352.0,13684457.0,942159.0,6671487.0,6070811.0,13936240.0,6380643.0,...,105469.0,1422671.0,589050.0,2065700.0,215.763583,263.639394,207.297609,255.226635,235.094096,235.094096
220,1996_HAWAII,HAWAII,1996,187104.0,1199019.0,91391.0,1079097.0,28531.0,1231550.0,671105.0,...,10466.0,119309.0,51427.0,171430.0,214.965562,262.130082,199.701682,260.800208,233.380993,233.380993
224,1996_IOWA,IOWA,1996,502343.0,3258040.0,147878.0,1486426.0,1623736.0,3284185.0,1679091.0,...,35650.0,292987.0,154912.0,452941.0,229.126198,283.986369,223.045210,271.566770,253.292493,253.292493


### 2) What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [21]:
education_df[['TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE',
             'CAPITAL_OUTLAY_EXPENDITURE','AVG_SCORE']].corr().T

,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,AVG_SCORE
TOTAL_EXPENDITURE,1.000000,0.990367,0.993535,0.944865,0.929195,0.074818
INSTRUCTION_EXPENDITURE,0.990367,1.000000,0.975915,0.904714,0.888705,0.088776
SUPPORT_SERVICES_EXPENDITURE,0.993535,0.975915,1.000000,0.951841,0.915248,0.078763
OTHER_EXPENDITURE,0.944865,0.904714,0.951841,1.000000,0.920591,0.004562
CAPITAL_OUTLAY_EXPENDITURE,0.929195,0.888705,0.915248,0.920591,1.000000,-0.004219
AVG_SCORE,0.074818,0.088776,0.078763,0.004562,-0.004219,1.000000


Instruction expenditure shows the most correlation with the Avg Scores

### 3) Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [34]:
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA


#Creating the 4 features we want
features=['TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE',
             'CAPITAL_OUTLAY_EXPENDITURE']

#Separating the 4 features we want
x = education_df.loc[:,features].values

#Separating the target variable
target = education_df.loc[:,['AVG_SCORE']].values

#Standardizing the features
x=StandardScaler().fit_transform(x)


sklearn_pca = PCA(n_components=4)
education_df['pca']=sklearn_pca.fit_transform(x)

print('The percentage of total variance in the dataset explained by each component from Sklearn PCA.\n', 
      sklearn_pca.explained_variance_ratio_
)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.95343764 0.02791759 0.01578554 0.00275496]


### 4) What is the correlation between the overall score variable and the 1st principal component?


In [37]:
education_df[['pca','TOTAL_EXPENDITURE', 'INSTRUCTION_EXPENDITURE','SUPPORT_SERVICES_EXPENDITURE','OTHER_EXPENDITURE',
             'CAPITAL_OUTLAY_EXPENDITURE','AVG_SCORE']].corr()

,pca,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE,AVG_SCORE
pca,1.000000,0.995346,0.975405,0.991011,0.967107,0.952713,0.050241
TOTAL_EXPENDITURE,0.995346,1.000000,0.990367,0.993535,0.944865,0.929195,0.074818
INSTRUCTION_EXPENDITURE,0.975405,0.990367,1.000000,0.975915,0.904714,0.888705,0.088776
SUPPORT_SERVICES_EXPENDITURE,0.991011,0.993535,0.975915,1.000000,0.951841,0.915248,0.078763
OTHER_EXPENDITURE,0.967107,0.944865,0.904714,0.951841,1.000000,0.920591,0.004562
CAPITAL_OUTLAY_EXPENDITURE,0.952713,0.929195,0.888705,0.915248,0.920591,1.000000,-0.004219
AVG_SCORE,0.050241,0.074818,0.088776,0.078763,0.004562,-0.004219,1.000000


### 5) If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

The first principal component explains 95% of the variance and the difference in correlation between this feature and the expenditure variables is not much different (range of -.004 to 07) compared to correlation of .05